# LlamaIndex ReAct Agent

1. ReAct Agent using simple calculator tools.
2. ReAct Agent using QueryEngine (RAG) tools.

https://docs.llamaindex.ai/en/stable/examples/agent/react_agent/

## Baseline

In [1]:
from llama_index.llms.openai import OpenAI

llm = OpenAI(model="gpt-3.5-turbo")
llm.complete("What is 20+(2*4)? Calculate step by step")

CompletionResponse(text='20 + (2*4) = 20 + 8 = 28', additional_kwargs={}, raw={'id': 'chatcmpl-9jhiMmE1DPmDMBZA3lYYYMxgQZvDf', 'choices': [Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='20 + (2*4) = 20 + 8 = 28', role='assistant', function_call=None, tool_calls=None))], 'created': 1720679826, 'model': 'gpt-3.5-turbo-0125', 'object': 'chat.completion', 'service_tier': None, 'system_fingerprint': None, 'usage': CompletionUsage(completion_tokens=16, prompt_tokens=20, total_tokens=36)}, logprobs=None, delta=None)

It does pretty well on this simple example, but we are kinda rolling the dice and hoping it gives us the right answer.

Instead, let's try it the ReAct way.

## ReAct with a Calculator

In [9]:
from llama_index.core.agent import ReActAgent
from llama_index.llms.openai import OpenAI
from llama_index.core.llms import ChatMessage
from llama_index.core.tools import BaseTool, FunctionTool

llm = OpenAI(model="gpt-4")

In [10]:
def multiply(a: int, b: int) -> int:
    """Multiply two integers and returns the result integer"""
    return a * b


multiply_tool = FunctionTool.from_defaults(fn=multiply)

def add(a: int, b: int) -> int:
    """Add two integers and returns the result integer"""
    return a + b

add_tool = FunctionTool.from_defaults(fn=add)

In [11]:
agent = ReActAgent.from_tools([multiply_tool, add_tool], llm=llm, verbose=True)
response = agent.chat("What is 20+(2x4)? Calculate step by step.")

print(response)

> Running step 7eb0c6ef-994a-4d25-909b-975a5cca2a5f. Step input: What is 20+(2x4)? Calculate step by step.
Thought: The user wants to calculate the expression 20+(2x4) step by step. According to the order of operations (BIDMAS/BODMAS), multiplication should be performed before addition. So, I will first calculate 2x4 using the 'multiply' tool.
Action: multiply
Action Input: {'a': 2, 'b': 4}
Observation: 8
> Running step 1c2f9a86-35dd-466c-ab40-66771fda2236. Step input: None
Thought: The multiplication result is 8. Now, I need to add this result to 20. I will use the 'add' tool for this.
Action: add
Action Input: {'a': 20, 'b': 8}
Observation: 28
> Running step 41158437-d23e-4980-b6db-dea6196fdaf4. Step input: None
Thought: I can answer without using any more tools. I'll use the user's language to answer.
Answer: The result of the calculation 20+(2x4) is 28.
The result of the calculation 20+(2x4) is 28.


In [12]:
prompt_dict = agent.get_prompts()
for k, v in prompt_dict.items():
    print(f"Prompt: {k}\n\nValue: {v.template}")

Prompt: agent_worker:system_prompt

Value: You are designed to help with a variety of tasks, from answering questions to providing summaries to other types of analyses.

## Tools

You have access to a wide variety of tools. You are responsible for using the tools in any sequence you deem appropriate to complete the task at hand.
This may require breaking the task into subtasks and using different tools to complete each subtask.

You have access to the following tools:
{tool_desc}


## Output Format

Please answer in the same language as the question and use the following format:

```
Thought: The current language of the user is: (user's language). I need to use a tool to help me answer the question.
Action: tool name (one of {tool_names}) if using a tool.
Action Input: the input to the tool, in a JSON format representing the kwargs (e.g. {{"input": "hello world", "num_beams": 5}})
```

Please ALWAYS start with a Thought.

Please use a valid JSON format for the Action Input. Do NOT do t

## Querying an API

In [13]:
from llama_index.core.tools.ondemand_loader_tool import OnDemandLoaderTool
from llama_index.readers.wikipedia import WikipediaReader
from typing import List

from pydantic import BaseModel

In [14]:
reader = WikipediaReader()

In [18]:
wikipedia_tool = OnDemandLoaderTool.from_defaults(
    reader,
    name="wikipedia_tool",
    description="A tool for loading and querying articles from Wikipedia",
)

In [19]:
wikipedia_tool(["Snorlax"], query_str="In what year did Snorlax debut?")

ToolOutput(content='Snorlax debuted in the Game Boy video game Pokémon Red and Blue, which was released in 1996.', tool_name='wikipedia_tool', raw_input={'query': 'In what year did Snorlax debut?'}, raw_output=Response(response='Snorlax debuted in the Game Boy video game Pokémon Red and Blue, which was released in 1996.', source_nodes=[NodeWithScore(node=TextNode(id_='0154872d-26ac-410c-b7f3-e39cf8ff0da7', embedding=None, metadata={}, excluded_embed_metadata_keys=[], excluded_llm_metadata_keys=[], relationships={<NodeRelationship.SOURCE: '1'>: RelatedNodeInfo(node_id='25824540', node_type=<ObjectType.DOCUMENT: '4'>, metadata={}, hash='6431e4ebddf77c17d4bfa7d5cd8514c408b204125ca4c97b30e9547b3159daae'), <NodeRelationship.PREVIOUS: '2'>: RelatedNodeInfo(node_id='80c307e6-397b-4855-b8c6-a384768d5a0a', node_type=<ObjectType.TEXT: '1'>, metadata={}, hash='19e5e534742de13802abd04e88607185208df810e3f2a9ede0f4a46e4dcce663')}, text='=== In other media ===\nA notable Snorlax appears in the Pokémo

In [20]:
agent = ReActAgent.from_tools([wikipedia_tool], llm=llm, verbose=True)
response = agent.chat("In what year did Snorlax debut?")

> Running step d256bc1c-63b0-45d9-9425-3c0aba0f7c29. Step input: In what year did Snorlax debut?
Thought: Snorlax is a character from the Pokémon franchise. I can find the debut year of Snorlax by querying the Wikipedia page for Snorlax.
Action: wikipedia_tool
Action Input: {'pages': ['Snorlax']}
Observation: Error: Missing query_str in kwargs with parameter name: query_str
> Running step 790de42e-045d-4ac8-96f8-488f5d555060. Step input: None
Thought: I made a mistake by not including a query string in my tool input. I need to specify what information I'm looking for on the Snorlax Wikipedia page.
Action: wikipedia_tool
Action Input: {'pages': ['Snorlax'], 'query_str': 'first appeared'}
Observation: Snorlax first appeared in the Game Boy video game Pokémon Red and Blue.
> Running step 6532b3bb-8896-493c-852b-110a989839fe. Step input: None
Thought: Snorlax first appeared in the Pokémon Red and Blue video games. I need to find out when these games were released to determine the debut yea

Pasting an example directly into ChatGPT

```
Context information is below.
---------------------
Snorlax debuted in the Game Boy video game Pokémon Red and Blue, which was released in 1996.
---------------------
Given the context information and not prior knowledge, answer the query.
Query: "In what year did Snorlax debut
Answer:  
```
> Snorlax debuted in 1996.

```
Context information is below.
---------------------
Snotlax debuted in the Game Boy video game Pokémon Orange and Blue, which was released in 2145.
---------------------
Given the context information and not prior knowledge, answer the query.
Query: In what year did Psyduck debut
Answer:
```
> Based on the context information provided, it states that Snotlax debuted in the Game Boy video game "Pokémon Orange and Blue" in 2145. However, there is no information about the debut year of Psyduck within the provided context.